# Upload files in Google Colab
If you are running this Jupyter Notebook on Google Colab, run this cell to upload the data files (train_inputs.csv, train_targets.csv, test_inputs.csv, test_targets.csv) in the colab virtual machine.  You will be prompted to select files that you would like to upload. 

If you are running this Jupyter Notebook on your computer, you do not need to run this cell.

In [ ]:
from google.colab import files
uploaded = files.upload()
%ls

# Import libraries 
Do not use any other Python library.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Function: load_linear_regression_data

This function loads the data for Linear Regression from a local drive into RAM

Outputs:

*   **train_inputs**: numpy array of N training data points x M features
*   **train_targets**: numpy array of N training targets
*   **test_inputs**: numpy array of N' test data points x M features
*   **test_targets**: numpy array of N' test targets

In [3]:
def load_linear_regression_data():
    test_inputs = np.genfromtxt('test_inputs.csv', delimiter=',')
    test_targets = np.genfromtxt('test_targets.csv', delimiter=',')
    train_inputs = np.genfromtxt('train_inputs.csv', delimiter=',')
    train_targets = np.genfromtxt('train_targets.csv', delimiter=',')
    return train_inputs, train_targets, test_inputs, test_targets


# Function: predict_linear_regression

This function uses a vector of weights to make predictions for a set of inputs.

Inputs:
*   **inputs**: matrix of input data points for which we want to make a prediction (numpy array of N data points x M+1 features)
*   **weights**: vector of weights (numpy array of M+1 weights)

Output:
*   **predicted_values**: vector of predicted values (numpy array of N floats)

In [33]:
def predict_linear_regression(inputs, weights):
    # The formula followed by this function : (predicted value = feature1 * weight1 + feature2 * weight2 + .....), where inputs is a set of inputs data (x1, x2, etc), weights is a vector of the slopes & intercepts (w0, w1, etc) and predicted_values is a vector of the response variables (y values)
    # Since the number of features = number of weights, and inputs shape is (200,2), the number of features is 2, meaning the number of weights is 2 as well
    num_of_datapts = len(inputs)
    predicted_values = np.zeros(inputs.shape[0])
    # iterate through the inputs array & use the formular to add values to the predicted_values array, using formula: predicted_value = input0*weight0 + input1*weight1
    for i in range(0, num_of_datapts):
        predicted_values.append((inputs[i][0]*weights[0]+inputs[i][1]*weights[1]))
    return predicted_values

# Function eval_linear_regression

This function evaluates a set of predictions by computing the mean squared error with respect to the targets

Inputs:
*   **inputs**: matrix of input data points for which we will evaluate the predictions (numpy array of N data points x M+1 features)
*   **weights**: vector of weights (numpy array of M+1 weights)
*   **targets**: vector of targets associated with the inputs (numpy array of N targets)

Output:
*   **mean_squared_error**: mean squared error between the predicted values and the targets (scalar)

In [36]:
def eval_linear_regression(inputs, weights, targets):
    # The Mean Squared Error formula is : 1/N *SUM(targets - predicted_values)^2
    # Call predict_liner_regression function to gather the estimated values
    predicted_values = predict_linear_regression(inputs, weights)
    # Get the total number of datapoints
    mean_squared_error = 0
    for i in range (0,N-1):
        # Iterate through both the predicted values & actual values (target), adding the square of the difference
        mean_squared_error += (targets[i]-predicted_values[i])**2
    # then multiply the mean squared error by 1/N
    mean_squared_error = mean_squared_error * (1/N)
    return mean_squared_error

# Function train_linear_regression

This function optimizes a set of weights for linear regression based on a training set

Inputs:
*   **train_inputs**: matrix of input training points (numpy array of N data points x M+1 features)
*   **train_targets**: vector of targets associated with the inputs (numpy array of N targets)
*   **lambda_hyperparam**: lambda hyperparameter used to adjust the importance of the regularizer (scalar)

Output:
*   **weights**: vector of weights that have been optimized (numpy array of M+1 weights)



In [76]:
def train_linear_regression(train_inputs, train_targets, lambda_hyperparam):
    weights = np.zeros(train_inputs.shape[1])
    # The formula for the weights is: weight = ((A +(lambda)*I)^-1) * b
    # A = SUM(Xn * Xn't), where Xn is the datapoint in train_inputs and Xn't is the datapoint in the transpose of train_inputs
    # We can start by finding the transpose of train_inputs matrix & finding A, which will be the product of train_input & it's transpose
    train_inputs_transpose = train_inputs.transpose()
    A = np.matmul(train_inputs_transpose,train_inputs)
    # Next, we'll have to calculate b, which is SUM(Xn * tn), where tn is the train_targets datapoints
    b = np.matmul(train_targets,train_inputs)
    # Next, we'll have to add I * λ to A, Where I is the identity matrix of size A.shape[0] and λ is lambda_hyperparam
    I = np.identity(A.shape[0])
    I = I * lambda_hyperparam
    A = A + I
    # Next, find the inverse of (A + Iλ)
    A_inverse = np.linalg.inv(A)
    # Finally, we compute the weight by multiplying A_inverse with b
    weights = np.matmul(A_inverse,b)
    return weights

# Function cross_validation_linear_regression

This function performs k-fold cross validation to determine the best lambda hyperparameter in linear regression

Inputs:
*   **k_folds**: # of folds in cross-validation (integer)
*   **hyperparameters**: list of hyperparameters where each hyperparameter is a different lambda value (list of floats)
*   **inputs**: matrix of input points (numpy array of N data points by M+1 features)
*   **targets**: vector of targets associated with the inputs (numpy array of N targets)

Outputs:
*   **best_hyperparam**: best lambda value for linear regression (float)
*   **best_mean_squared_error**: mean squared error achieved with best_hyperparam (float)
*   **mean_squared_errors**: vector of mean squared errors for the corresponding hyperparameters (numpy array of floats)



In [81]:
a = [[1],[2],[3],[4]]
for i in range(0,3):
    a[i].append(i)
a

[[1, 0], [2, 1], [3, 2], [4]]

In [82]:
def cross_validation_linear_regression(k_folds, hyperparameters, inputs, targets):
    mean_squared_errors = np.zeros(len(hyperparameters))
    num_of_cols = len(inputs[0])
    # First, join inputs & labels prior to shuffling, in order to keep results correlated to input data & then shuffle
    train_data = np.append(inputs, targets, axis=1)
    np.random.shuffle(train_data) 
    # Then split train_data back into inputs & targets
    train_inputs = train_data[:,:num_of_cols]
    train_targets = train_data[:,num_of_cols]
    for i in hyperparameters:
        # Next, we will iterate through the hyperparameters array and find the weights for each value of lambda
        weights = train_linear_regression(train_inputs, train_targets, train_targets)
        # Then we will append each MSE value to mean_squared_errors
        mean_squared_errors.append(eval_linear_regression(train_inputs,weights,i))
    # Afterwards, we have to find the best MSE, that being the lowest MSE
    best_hyperparam = min(mean_squared_errors)
    # Since we iterate through the hyperparameters & add the MSE values respectively, the index of the mininum MSE is the index of the corresponding best hyperparamter in hyperparameters
    hyperparam_index = best_accuracies.index(best_accuracy).astype('int')
    best_mean_squared_error = mean_squared_errors[hyperparam_index]
    return best_hyperparam, best_mean_squared_error, mean_squared_errors

# Function: plot_linear_regression_mean_squared_errors

Function that plots the mean squared errors for different lambda values (hyperparameters) in linear regression based on cross validation

Inputs:
*   **mean_squared_errors**: vector of mean squared errors for the corresponding hyperparameters (numpy array of floats)
*   **hyperparams**: list of hyperparameters where each hyperparameter is a different lambda value (list of floats)

In [8]:
def plot_linear_regression_mean_squared_errors(mean_squared_errors,hyperparams):
    plt.plot(hyperparams,mean_squared_errors)
    plt.ylabel('mean squared error')
    plt.xlabel('lambda')
    plt.show()

# Main Linear Regression code

Load data.
Use k-fold cross validation to find the best lambda value for linear regression.
Plot mean squared errors for different lambda values.
Test linear regression with the best lambda value.

In [ ]:
# load data
train_inputs, train_targets, test_inputs, test_targets = load_linear_regression_data()

# add 1 at the end of each data point
train_inputs = np.concatenate((train_inputs,np.ones((train_inputs.shape[0],1))),1)
test_inputs = np.concatenate((test_inputs,np.ones((test_inputs.shape[0],1))),1)

# lambda values to be evaluated by cross validation
hyperparams = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0]
k_folds = 10
best_lambda, best_mean_squared_error, mean_squared_errors = cross_validation_linear_regression(k_folds,hyperparams,train_inputs,train_targets)

# plot results
plot_linear_regression_mean_squared_errors(mean_squared_errors,hyperparams)
print('best lambda: ' + str (best_lambda))
print('best cross validation mean squared error: ' + str(best_mean_squared_error))

# train and evaluate with best lambda
weights = train_linear_regression(train_inputs,train_targets,best_lambda)
mean_squared_error = eval_linear_regression(test_inputs, weights, test_targets)
print('test mean squared error: ' + str(mean_squared_error))

